In [2]:
!pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd

In [4]:
kundendaten = pd.read_csv("data/Kundendaten.csv", sep=";", encoding="latin1")
produktdaten = pd.read_csv("data/Produktdaten.csv", sep=";", encoding="latin1")
transaktionsdaten = pd.read_csv("data/Transaktionsdaten.csv", sep=";", encoding="latin1")

In [5]:
kundendaten = kundendaten.drop(columns=["Datensatz_ID"])
kundendaten = kundendaten.dropna()
kundendaten = kundendaten.drop_duplicates()
kundendaten["Kunde_Nr"] = kundendaten["Kunde_Nr"].astype(int)

kundendaten["Beruf"] = kundendaten["Beruf"].str.replace(r"\*In|\*Innen", "", case=False, regex=True)
kundendaten["Beruf"] = kundendaten["Beruf"].str.replace(r"in", "", case=True, regex=True)
kundendaten["Beruf"] = kundendaten["Beruf"].str.replace(r"frau", "mann", case=True, regex=True)
kundendaten["Beruf"] = kundendaten["Beruf"].str.replace(r"stud.*", "Student", case=False, regex=True)
kundendaten["Beruf"] = kundendaten["Beruf"].str.replace(r"Biolog.*", "Biologe", case=False, regex=True)
kundendaten["Beruf"] = kundendaten["Beruf"].str.replace(r"Mama|Papa", "Eltern", case=False, regex=True)
kundendaten["Beruf"] = kundendaten["Beruf"].str.replace(r"Oma|Opa", "Großeltern", case=False, regex=True)
kundendaten["Beruf"] = kundendaten["Beruf"].str.replace(r"Rechtsanw.*", "Rechtsanwalt", case=False, regex=True)

kundendaten["Geschlecht"] = kundendaten["Geschlecht"].str.replace(r"m.*|0", "männlich", case=False, regex=True)
kundendaten["Geschlecht"] = kundendaten["Geschlecht"].str.replace(r"w.*|f.*|1", "weiblich", case=False, regex=True)

In [6]:
produktdaten = produktdaten.dropna()
produktdaten["Buch_ID"] = produktdaten["Buch_ID"].astype(int)
produktdaten["Preis"] = produktdaten["Preis"].str.replace(".","", regex=False).str.replace(",",".", regex=False).astype(float)
produktdaten = produktdaten[(produktdaten["Preis"] > 0) & (produktdaten["Preis"] < 1000)]
produktdaten["Buch_typ"] = produktdaten["Buch_typ"].str.replace("hc","hard cover").replace("pb","paperback").replace("Taschenbuch","paperback")

for i in produktdaten.index:
    titel = produktdaten.loc[i,"Titel"]
    if "\n" in titel:
        produktdaten.loc[i,"Titel"] = titel.split("\n")[0]
    if titel[0] == " ":
        produktdaten.loc[i,"Titel"] = titel[1:]

In [7]:
transaktionsdaten = transaktionsdaten.dropna()
transaktionsdaten = transaktionsdaten.drop(columns=["Datensatz_ID"])
transaktionsdaten["Kauf_Datum"] = transaktionsdaten["Kauf_Datum"].str.replace(" 00:00:00", "").str.replace("1023","2023")
transaktionsdaten["Kauf_Datum"] = pd.to_datetime(transaktionsdaten["Kauf_Datum"], format="%d.%m.%Y", dayfirst=True)

In [8]:
merged_data = pd.merge(transaktionsdaten, kundendaten, left_on="Kunden_Nr", right_on="Kunde_Nr")
merged_data = pd.merge(merged_data, produktdaten, on="Buch_ID")
merged_data = merged_data.drop(columns=["Kunde_Nr"])

In [9]:
merged_data[merged_data["Alter"] < 18].Beruf.value_counts()

Beruf
Hausmann        47
Eltern          44
Student         44
Doktorand        9
Rechtsanwalt     9
Richter          2
Elektroniker     2
Biologe          1
Name: count, dtype: int64

In [10]:
merged_data = merged_data[merged_data["Alter"] > 0]

In [11]:
merged_data

,Kauf_Datum,Kunden_Nr,Buch_ID,Kauf_Typ,Kunde_Name,Wohnort_Bezirk,Beruf,Geschlecht,Religionszugehörigkeit,Alter,Titel,Autor,Preis,Buch_typ
0,2023-01-12,23393,179,spontan,Thomas Schmitt,4,Rentner,männlich,konfessionslos,72,Mittsommermord,Henning Mankell,21.0,paperback
1,2023-03-25,23311,179,spontan,Gudrun Wagner,4,Pensionär,weiblich,muslimisch,68,Mittsommermord,Henning Mankell,21.0,paperback
2,2023-02-28,23327,179,spontan,Christiane Maier,8,Pensionär,weiblich,evangelisch,70,Mittsommermord,Henning Mankell,21.0,paperback
3,2023-05-11,23331,179,spontan,Fritz Yilmaz,4,Großeltern,männlich,katholisch,79,Mittsommermord,Henning Mankell,21.0,paperback
4,2023-11-11,23372,179,spontan,Michael Schmid,4,Großeltern,männlich,katholisch,74,Mittsommermord,Henning Mankell,21.0,paperback
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939,2023-09-23,23434,183,vorbestellt,Franz Fassbender,4,Eltern,männlich,evangelisch,32,Geschichte Mittelstufe 8,Hilde Basler,26.0,hard cover
2940,2023-09-26,23471,183,vorbestellt,Kurt Zaiser,4,Eltern,männlich,evangelisch,40,Geschichte Mittelstufe 8,Hilde Basler,26.0,hard cover
2941,2023-09-27,23477,153,vorbestellt,Franz Huber,4,Hausmann,männlich,evangelisch,52,Mathematik Mittelstufe 8,Erwin Dridger,23.0,hard cover
2942,2023-09-07,23489,153,vorbestellt,Anne Zaiser,4,Eltern,weiblich,konfessionslos,58,Mathematik Mittelstufe 8,Erwin Dridger,23.0,hard cover


In [12]:
merged_data.to_pickle("data/merged_data.pkl")
kundendaten.to_pickle("data/kundendaten.pkl")
produktdaten.to_pickle("data/produktdaten.pkl")
transaktionsdaten.to_pickle("data/transaktionsdaten.pkl")

In [14]:
merged_data["Beruf"].value_counts()

Beruf
Student                   701
Hausmann                  448
Eltern                    397
Frührentner               319
Pensionär                 271
Großeltern                146
Doktorand                 110
Rentner                    79
Ruheständler               71
Elektroniker               52
nicht mehr berufstätig     51
Vorruheständler            49
Bäcker                     35
Chemiker                   33
Betriebswirt               30
Rechtsanwalt               30
Bankkaufmann               26
Klemptner                  20
IT-Fachmann                16
Metzger                    15
Richter                    13
Biologe                    10
Steuerberater               4
Altenpfleger                3
Name: count, dtype: int64